In [3]:
import pandas as pd
import numpy as np

# sklearn imports for modeling part
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions

from sklearn.metrics import confusion_matrix

In [4]:
df = pd.read_csv('kindle_data.csv', low_memory=False)

In [5]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"07 3, 2014",A2LSKD2H9U8N0J,B000FA5KK0,{'Format:': ' Kindle Edition'},sandra sue marsolek,"pretty good story, a little exaggerated, but I...",pretty good story,1404345600,NaN,NaN
1,5.0,True,"05 26, 2014",A2QP13XTJND1QS,B000FA5KK0,{'Format:': ' Kindle Edition'},Tpl,"If you've read other max brand westerns, you k...",A very good book,1401062400,NaN,NaN
2,5.0,True,"09 16, 2016",A8WQ7MAG3HFOZ,B000FA5KK0,{'Format:': ' Kindle Edition'},Alverne F. Anderson,"Love Max, always a fun twist",Five Stars,1473984000,NaN,NaN
3,5.0,True,"03 3, 2016",A1E0MODSRYP7O,B000FA5KK0,{'Format:': ' Kindle Edition'},Jeff,"As usual for him, a good book",a good,1456963200,NaN,NaN
4,5.0,True,"09 10, 2015",AYUTCGVSM1H7T,B000FA5KK0,{'Format:': ' Kindle Edition'},DEHS - EddyRapcon,MB is one of the original western writers and ...,A Western,1441843200,2,NaN


In [10]:
df['reviewerID'].value_counts().head(5)

A1JLU5H1CCENWX    1377
A328S9RN3U5M68    1234
A13QTZ8CIMHHG4    1011
A3GWE80SUGORJD    1007
A2YJ8VP1SSHJ7      931
Name: reviewerID, dtype: int64

In [6]:
df1=df.loc[df['reviewerID']=='A1JLU5H1CCENWX']
df2=df.loc[df['reviewerID']=='A328S9RN3U5M68']
df3=df.loc[df['reviewerID']=='A13QTZ8CIMHHG4']
df4=df.loc[df['reviewerID']=='A3GWE80SUGORJD']
df5=df.loc[df['reviewerID']=='A2YJ8VP1SSHJ7']

In [7]:
df_merged=pd.concat([df1, df2,df3,df4,df5])

In [8]:
df_merged.reset_index(inplace=True)

In [18]:
df_merged

,index,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5570,5.0,True,"11 3, 2013",A1JLU5H1CCENWX,B001892DGG,{'Format:': ' Kindle Edition'},Heather- A Crazy Vermonter&#039;s Book Reviews,"Max holy male hotness! ""hot damn, Max thought,...",Max,1383436800,NaN,NaN
1,6147,5.0,True,"11 3, 2013",A1JLU5H1CCENWX,B001CN45ZA,{'Format:': ' Kindle Edition'},Heather- A Crazy Vermonter&#039;s Book Reviews,Simon he may be the beta of the Pride but that...,Simon,1383436800,NaN,NaN
2,8060,5.0,True,"11 3, 2013",A1JLU5H1CCENWX,B0026OQYYO,{'Format:': ' Kindle Edition'},Heather- A Crazy Vermonter&#039;s Book Reviews,"Rick he loves his mate and he cares for her, ""...",Rick,1383436800,NaN,NaN
3,8431,4.0,True,"11 3, 2013",A1JLU5H1CCENWX,B002A4MICW,{'Format:': ' Kindle Edition'},Heather- A Crazy Vermonter&#039;s Book Reviews,"Adrian is such a damn goofball, ""but my mate i...",Adrian,1383436800,NaN,NaN
4,9889,4.0,True,"11 3, 2013",A1JLU5H1CCENWX,B002VFPS9A,{'Format:': ' Kindle Edition'},Heather- A Crazy Vermonter&#039;s Book Reviews,"Gabe, he's not much for another man touching w...",Gabe,1383436800,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,2207737,5.0,False,"08 6, 2017",A2YJ8VP1SSHJ7,B01FWZZ55W,{'Format:': ' Kindle Edition'},Kindle Customer,This was extremely good to read. I know I read...,Fugitive,1501977600,NaN,NaN
5556,2213697,5.0,False,"09 9, 2016",A2YJ8VP1SSHJ7,B01GJ3KU8I,{'Format:': ' Kindle Edition'},Kindle Customer,I'm glad Dominic took over although he didn't ...,Guardian Bears: Lucas,1473379200,NaN,NaN
5557,2215798,5.0,False,"09 22, 2016",A2YJ8VP1SSHJ7,B01GSQLHWE,{'Format:': ' Kindle Edition'},Kindle Customer,These were just so perfect. I just wish they h...,Zebir,1474502400,NaN,NaN
5558,2217213,4.0,False,"06 27, 2016",A2YJ8VP1SSHJ7,B01GYZR534,{'Format:': ' Kindle Edition'},Kindle Customer,I wish they had caught the scientist so they c...,Alien UnSealed,1466985600,NaN,NaN


In [23]:
df_merged.groupby('asin').count()

,index,overall,verified,reviewTime,reviewerID,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
asin,,,,,,,,,,,,
B000VKVZQM,1,1,1,1,1,1,1,1,1,1,1,0
B0012W11D0,2,2,2,2,2,2,2,2,2,2,0,0
B001892DGG,2,2,2,2,2,2,2,2,2,2,0,0
B001CN45ZA,1,1,1,1,1,1,1,1,1,1,0,0
B001EO181Q,1,1,1,1,1,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
B01HFNB72O,1,1,1,1,1,1,1,1,1,1,0,0
B01HH2TGFI,1,1,1,1,1,1,1,1,1,1,0,0
B01HHJIUGW,1,1,1,1,1,1,1,1,1,1,0,0


In [9]:
from nltk.corpus import stopwords

In [10]:
def clear(city):
    #1
    city = city.lower()
    #2
    city = city.split()
    #3
    city_keywords = [word for word in city if word not in stopwords.words('english')]

    merged_city = " ".join(city_keywords)
    return merged_city

for index, row in df_merged.iterrows():
    clear_desc = clear(row['reviewText'])
    df_merged.at[index, 'reviewText'] = clear_desc


In [19]:
import re, math
from collections import Counter

# class CosineSimilarity:
#     def __init__(self):
#         print("Cosine Similarity initialized")

#     def cosine_similarity_of(text1, text2):
#         #get words first
#         first = re.compile(r"[\w']+").findall(text1)
#         second = re.compile(r"[\w']+").findall(text2)

#         #get dictionary with each word and count.
#         vector1 = Counter(first)
#         vector2 = Counter(second)

#         #convert vectors to set to find common words as intersection
#         common = set(vector1.keys()).intersection(set(vector2.keys()))

#         dot_product = 0.0

#         for i in common:
#             #get amount of each common word for both vectors and multiply them then add them together.
#             dot_product += vector1[i] * vector2[i]

#         squared_sum_vector1 = 0.0
#         squared_sum_vector2 = 0.0

#         #get squared sum values of word counts from each vector.
#         for i in vector1.keys():
#             squared_sum_vector1 += vector1[i]**2

#         for i in vector2.keys():
#             squared_sum_vector2 += vector2[i]**2

#         #calculate magnitude with squared sums.
#         magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

#         if not magnitude:
#             return 0.0
#         else:
#             return float(dot_product) / magnitude


In [20]:
def cosine_similarity_of(text1, text2):
        #get words first
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        #get dictionary with each word and count.
        vector1 = Counter(first)
        vector2 = Counter(second)

        #convert vectors to set to find common words as intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            #get amount of each common word for both vectors and multiply them then add them together.
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        #get squared sum values of word counts from each vector.
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        #calculate magnitude with squared sums.
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
            return 0.0
        else:
            return float(dot_product) / magnitude


In [21]:
text1 = df_merged['reviewText'][0]

In [22]:
text2 = df_merged['reviewText'][1]

In [23]:
cosine_similarity_of(text1, text2)

0.04721921646498856

In [24]:
import re
import sklearn.metrics.pairwise as pw
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

In [122]:
def user_based_recom(input_dataframe,input_user_id):    
    
    pivot_user_based = pd.pivot_table(input_dataframe, index='asin', columns=['reviewerID'], values='overall').T
    sparse_pivot_ub = sparse.csr_matrix(pivot_user_based.fillna(0))
    user_recomm = pw.cosine_similarity(sparse_pivot_ub)
    user_recomm_df = pd.DataFrame(user_recomm,columns=pivot_user_based.index.values,
                 index=pivot_user_based.index.values)
    ## Item Rating Based Cosine Similarity
    usr_cosine_df = pd.DataFrame(user_recomm_df[input_user_id].sort_values(ascending=False))
    usr_cosine_df.reset_index(level=0, inplace=True)
    usr_cosine_df.columns = ['reviewerID','cosine_sim']
    return usr_cosine_df

In [123]:
user_based_recom(df_merged,'A3GWE80SUGORJD')

,reviewerID,cosine_sim
0,A3GWE80SUGORJD,1.000000
1,A1JLU5H1CCENWX,0.233379
2,A13QTZ8CIMHHG4,0.010240
3,A2YJ8VP1SSHJ7,0.004719
4,A328S9RN3U5M68,0.003899


In [149]:
pivot_user_based = pd.pivot_table(df_merged, index='reviewerID', columns=['asin'], values='overall')
# pivot_user_based = insert_ratings(books_rated,pivot_user_based) #to add new user

In [150]:
user_sparse_pivot = sparse.csr_matrix(pivot_user_based.fillna(0))
user_recommender = pw.cosine_similarity(user_sparse_pivot)
user_recommender_df = pd.DataFrame(user_recommender, columns=pivot_user_based.index.values,index = pivot_user_based.index.values)

In [151]:
usr_cosine_df = pd.DataFrame(user_recommender_df['A13QTZ8CIMHHG4'].sort_values(ascending=False))
usr_cosine_df.reset_index(level=0, inplace=True)
usr_cosine_df.columns = ['asin','cosine_sim']
    ## 2 most similar users
similar_usr = list(usr_cosine_df['asin'][1:3].values)

In [152]:
similar_usr_df = pivot_user_based.T[['A13QTZ8CIMHHG4'] + similar_usr].fillna(0)
similar_usr_df['mean_rev'] = similar_usr_df[similar_usr].mean(numeric_only=True,axis=1)
similar_usr_df.sort_values('mean_rev', ascending=False,inplace = True)

In [153]:
similar_usr_df[similar_usr_df['A13QTZ8CIMHHG4']==0].head(5)

reviewerID,A13QTZ8CIMHHG4,A1JLU5H1CCENWX,A328S9RN3U5M68,mean_rev
asin,,,,
B00VD043F4,0.0,5.0,5.0,5.0
B00JJ5EV02,0.0,5.0,0.0,2.5
B00JTI8SR6,0.0,5.0,0.0,2.5
B0153KUUEC,0.0,5.0,0.0,2.5
B00JTIZV2Q,0.0,5.0,0.0,2.5


In [154]:
df_merged.loc[df_merged['asin']=='B00VD043F4']

,index,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
831,1066491,5.0,False,"09 29, 2015",A1JLU5H1CCENWX,B00VD043F4,{'Format:': ' Kindle Edition'},Heather- A Crazy Vermonter&#039;s Book Reviews,"clare needs perspective needs look casey it, ""...",Rory,1443484800,3,NaN
2442,2017339,5.0,True,"03 4, 2017",A328S9RN3U5M68,B00VD043F4,{'Format:': ' Kindle Edition'},Grady Harp,"washington, dc author sarah robinson earned ba...","Wouldnt dream of it, mhuirnn.",1488585600,9,NaN
4120,1066504,5.0,False,"09 29, 2015",A3GWE80SUGORJD,B00VD043F4,{'Format:': ' Kindle Edition'},Bette Hansen,really enjoyed one happy discovered new author...,Awesome,1443484800,7,NaN


# To add new user to dataframe

In [112]:
# books_rated = pd.read_excel("Book1.xlsx",index_col='Books')

In [127]:
# def insert_ratings(usr_rtg_dict,pivot_user_based):
#     """
#     Function to insert new row in pivot_user_based with book ratings created by a new user 
#     """
#     #loop throgh dictionary with ids of people who rated
#     for person_id in usr_rtg_dict:
#     #Loop through books that where rated
#         for book in books_rated.get(person_id):
#     #Inset ratings in main DF
#               pivot_user_based.loc[person_id,book] = usr_rtg_dict.get(person_id).get(book)

#     return pivot_user_based
